In [36]:
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

In [37]:
df = pd.read_csv('triplet.csv')

In [38]:
df['anchor'] = df['anchor'].apply(lambda x: torch.tensor(json.loads(x)))
df['pos'] = df['pos'].apply(lambda x: torch.tensor(json.loads(x)))
df['neg'] = df['neg'].apply(lambda x: torch.tensor(json.loads(x)))

df['distance_pos'] = df.apply(lambda row: 1.0 - F.cosine_similarity(row['anchor'], row['pos'], dim=0).item(), axis=1)
df['distance_neg'] = df.apply(lambda row: 1.0 - F.cosine_similarity(row['anchor'], row['neg'], dim=0).item() , axis=1)

In [39]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [40]:
df.head()

,anchor,pos,neg,distance_pos,distance_neg
0,"[tensor(0.0137), tensor(-0.0254), tensor(0.052...","[tensor(-0.0159), tensor(-0.0480), tensor(0.05...","[tensor(-0.0017), tensor(-0.0277), tensor(0.04...",0.227689,0.193694
1,"[tensor(-0.0219), tensor(-0.0300), tensor(0.04...","[tensor(-0.0159), tensor(-0.0480), tensor(0.05...","[tensor(0.0164), tensor(-0.0345), tensor(0.028...",0.253759,0.142795
2,"[tensor(0.0122), tensor(-0.0277), tensor(0.049...","[tensor(-0.0486), tensor(-0.0668), tensor(0.08...","[tensor(-0.0014), tensor(-0.0308), tensor(0.06...",0.307285,0.171227
3,"[tensor(-0.0180), tensor(-0.0444), tensor(0.03...","[tensor(-0.0486), tensor(-0.0668), tensor(0.08...","[tensor(-0.0326), tensor(-0.0536), tensor(0.05...",0.225380,0.198195
4,"[tensor(-0.0038), tensor(-0.0525), tensor(0.06...","[tensor(-0.0486), tensor(-0.0668), tensor(0.08...","[tensor(0.0042), tensor(-0.0550), tensor(0.018...",0.297195,0.161455


In [41]:
class LinearAdapter(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)
    
    def forward(self, x):
        return self.linear(x)

In [ ]:
load_dict = torch.load('adapter.pth')
loaded_adapter = LinearAdapter(768)
loaded_adapter.load_state_dict(load_dict['adapter_state_dict'])
loaded_adapter.eval()

/tmp/ipykernel_61186/3994794865.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_dict = torch.load('non_linear_adapter_30_epochs.pth')


<All keys matched successfully>

In [43]:
df['adapted_distance_pos'] = df.apply(lambda row: 1.0 - F.cosine_similarity(loaded_adapter(row['anchor']), loaded_adapter(row['pos']), dim=0).item(), axis=1)
df['adapted_distance_neg'] = df.apply(lambda row: 1.0 - F.cosine_similarity(loaded_adapter(row['anchor']), loaded_adapter(row['neg']), dim=0).item() , axis=1)

In [48]:
result = df[["distance_pos", "distance_neg", "adapted_distance_pos", "adapted_distance_neg"]]

In [52]:
result["res"] = result.apply(lambda row: 1 if row["adapted_distance_pos"] < row["adapted_distance_neg"] else 0, axis=1)

/tmp/ipykernel_61186/2526936484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["res"] = result.apply(lambda row: 1 if row["adapted_distance_pos"] < row["adapted_distance_neg"] else 0, axis=1)


In [53]:
result

,distance_pos,distance_neg,adapted_distance_pos,adapted_distance_neg,res
0,0.227689,0.193694,0.737273,1.829253,1
1,0.253759,0.142795,0.181191,1.394231,1
2,0.307285,0.171227,0.480764,1.759256,1
3,0.225380,0.198195,0.572937,1.661588,1
4,0.297195,0.161455,0.694121,1.839455,1
...,...,...,...,...,...
206,0.246686,0.114108,0.823098,1.875909,1
207,0.204091,0.124896,0.681165,1.762008,1
208,0.206771,0.140783,0.805243,1.889473,1
209,0.151370,0.140783,0.813292,1.889473,1


In [55]:
result["res"].value_counts()

res
1    211
Name: count, dtype: int64